# Complete AuDHD Correlation Study Workflow

This notebook demonstrates the complete pipeline from data loading to results visualization.

## Overview

1. Load multi-omics data
2. Preprocess and harmonize
3. Integrate modalities
4. Cluster samples
5. Validate clusters
6. Biological interpretation
7. Visualize results

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from audhd_correlation.data import load_multiomics, align_multiomics
from audhd_correlation.preprocess import preprocess_pipeline
from audhd_correlation.integrate import integrate_multiomics
from audhd_correlation.modeling import perform_clustering, create_embedding
from audhd_correlation.validation import validate_clusters
from audhd_correlation.biological import compute_cluster_signatures
from audhd_correlation.viz import plot_embedding, plot_heatmap

%matplotlib inline
sns.set_style('whitegrid')

## 1. Load Data

Load genomic, clinical, and metabolomic data.

In [ ]:
# Load data
data = load_multiomics(
    genomic_path="data/genomic/genotypes.vcf",
    clinical_path="data/clinical/phenotypes.csv",
    metabolomic_path="data/metabolomic/metabolites.csv"
)

print(f"Loaded {len(data)} modalities:")
for modality, df in data.items():
    print(f"  {modality}: {df.shape[0]} samples × {df.shape[1]} features")

In [ ]:
# Align samples across modalities
aligned_data = align_multiomics(data)

n_samples = len(aligned_data['genomic'])
print(f"After alignment: {n_samples} samples")

## 2. Preprocessing

Impute missing values, normalize, and apply quality control.

In [ ]:
# Preprocess each modality
preprocessed = preprocess_pipeline(
    aligned_data,
    impute_method='knn',
    scale_method='standard',
    batch_correct=True,
    batch_column='site'
)

print("Preprocessing complete:")
for modality, df in preprocessed.items():
    missing_pct = (df.isna().sum().sum() / df.size) * 100
    print(f"  {modality}: {missing_pct:.2f}% missing")

## 3. Integration

Integrate modalities using MOFA.

In [ ]:
# Integrate using MOFA
integrated = integrate_multiomics(
    preprocessed,
    method='mofa',
    n_factors=15
)

print(f"Integrated data shape: {integrated['factors'].shape}")
print(f"Variance explained: {integrated['variance_explained']:.1%}")

In [ ]:
# Plot variance explained per modality
var_df = pd.DataFrame(integrated['variance_per_modality'])
var_df.plot(kind='bar', figsize=(10, 5))
plt.title('Variance Explained per Modality')
plt.ylabel('R² Score')
plt.xlabel('Factor')
plt.tight_layout()
plt.show()

## 4. Clustering

Identify patient subtypes using HDBSCAN.

In [ ]:
# Perform clustering
cluster_result = perform_clustering(
    integrated['factors'],
    method='hdbscan',
    min_cluster_size=20
)

labels = cluster_result['labels']
n_clusters = cluster_result['n_clusters']

print(f"Found {n_clusters} clusters")
print(f"Cluster sizes: {np.bincount(labels[labels >= 0])}")
print(f"Noise points: {(labels == -1).sum()}")

In [ ]:
# Create UMAP embedding
embedding = create_embedding(
    integrated['factors'],
    method='umap',
    n_neighbors=15
)

# Plot clusters
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=labels,
    cmap='tab10',
    s=50,
    alpha=0.7
)
plt.colorbar(scatter, label='Cluster')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.title(f'Patient Clusters (n={n_clusters})')
plt.tight_layout()
plt.show()

## 5. Validation

Validate cluster quality and stability.

In [ ]:
# Compute validation metrics
validation = validate_clusters(
    integrated['factors'],
    labels,
    n_bootstrap=100
)

print("=== Validation Results ===")
print(f"Silhouette Score: {validation['silhouette']:.3f}")
print(f"Calinski-Harabasz: {validation['calinski_harabasz']:.1f}")
print(f"Davies-Bouldin: {validation['davies_bouldin']:.3f}")
print(f"Bootstrap Stability (ARI): {validation['stability_ari']:.3f}")

## 6. Clinical Characterization

Compare clusters by clinical variables.

In [ ]:
# Load clinical data
clinical = aligned_data['clinical']

# Add cluster labels
clinical['cluster'] = labels

# Remove noise points for analysis
clinical_clustered = clinical[clinical['cluster'] >= 0]

# Compare age across clusters
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Age
clinical_clustered.boxplot(column='age', by='cluster', ax=axes[0])
axes[0].set_title('Age by Cluster')
axes[0].set_xlabel('Cluster')
axes[0].set_ylabel('Age (years)')

# Severity score
clinical_clustered.boxplot(column='severity_score', by='cluster', ax=axes[1])
axes[1].set_title('Severity Score by Cluster')
axes[1].set_xlabel('Cluster')

# Diagnosis distribution
diag_counts = clinical_clustered.groupby(['cluster', 'diagnosis']).size().unstack(fill_value=0)
diag_counts.plot(kind='bar', ax=axes[2], stacked=True)
axes[2].set_title('Diagnosis Distribution by Cluster')
axes[2].set_xlabel('Cluster')
axes[2].set_ylabel('Count')
axes[2].legend(title='Diagnosis')

plt.tight_layout()
plt.show()

## 7. Biological Interpretation

Identify cluster signatures.

In [ ]:
# Compute cluster signatures
signatures = compute_cluster_signatures(
    preprocessed,
    labels,
    method='limma'
)

print("Cluster signatures:")
for cluster_id, sig in signatures.items():
    if cluster_id >= 0:  # Skip noise
        print(f"  Cluster {cluster_id}: {len(sig)} significant features")

## 8. Save Results

Export results for further analysis.

In [ ]:
# Save cluster assignments
results_df = pd.DataFrame({
    'sample_id': integrated['factors'].index,
    'cluster': labels,
    'umap1': embedding[:, 0],
    'umap2': embedding[:, 1]
})

results_df.to_csv('outputs/cluster_assignments.csv', index=False)

# Save validation metrics
import json
with open('outputs/validation_metrics.json', 'w') as f:
    json.dump(validation, f, indent=2)

print("Results saved to outputs/")

## Summary

This workflow demonstrates:

1. ✓ Loading and harmonizing multi-omics data
2. ✓ Preprocessing with imputation and normalization
3. ✓ Integration using MOFA
4. ✓ Clustering with HDBSCAN
5. ✓ Validation of cluster quality
6. ✓ Clinical characterization
7. ✓ Biological interpretation

## Next Steps

- Perform pathway enrichment analysis
- Create comprehensive report
- Validate on independent cohort